## Post disaster target's polygons alignment (Imposible)

### Metadatos relevantes de los json
#### Útil para calcular transformación
- "gsd":(Ground Sample Distance) Es la distancia en el suelo que cada píxel de la imagen representa. Metro por pixel.
- "off_nadir_angle": Es el ángulo entre la dirección de observación del sensor y la vertical hacia abajo (nadir).
- "target_azimuth": Es la dirección en la que el sensor de la imagen está apuntando, medida en grados en el plano horizontal.
- "original_width": Ancho original de la imagen del satélite
- "original_height": Alto original de la imagen del satélite
#### No útil para calcular transformación
"width": 1024,
"height": 1024
"pan_resolution" 
"sun_azimuth"
"sun_elevation"

In [ ]:
import json
import os
import numpy as np
from shapely.wkt import loads
from pyproj import Proj
from rasterio.transform import from_origin,Affine
import cv2
import pandas as pd
import math
from random import uniform

def find_corner_in_image(target_path: os.path):
    """
        Encuentra la esquina superior e inferior de un rectángulo que contiene todos los polígonos en la imagen.
        Devuelve las coordenadas en pixeles de la imagen.
    """
    target_img = cv2.imread(target_path)
    white_pixels = np.where(target_img > 0)
    return (min(white_pixels[1]),max(white_pixels[1]),min(white_pixels[0]),max(white_pixels[0]))

def get_affine(alpha,beta,scale):
    """
        Matriz que deforma un plano en 3 dimensiones. 
        alpha : diferencia entre angulo de off_nadir de pre con post
        beta : diferencia en angulo de target_azimuth de pre con post
        scale : escala que por la cual multiplicar GSD de pre para que sea igual a GSD de post
    """
    alpha = np.deg2rad(alpha)
    beta = np.deg2rad(beta)
    # Rotar con respecto al eje Z beta grados
    # Escalar x como y scale veces
    # Rota con respecto al eje x alpha grados
    #transform_matrix = np.array([[scale*np.cos(beta),-np.sin(beta)                   ,0             ],
    #                           [np.sin(beta)      ,scale*np.cos(beta)*np.cos(alpha),-np.sin(alpha)],
    #                           [0                 ,np.sin(alpha)                   ,np.cos(alpha) ]])
    transform_matrix = Affine(scale*np.cos(beta),-np.sin(beta),0,np.sin(beta),scale*np.cos(beta)*np.cos(alpha),-np.sin(alpha),0,np.sin(alpha),np.cos(alpha))
    return transform_matrix

def transform_polygons(geo_poly_list,target_path,pre_meta,post_meta):
    #print(post_meta["sat_nad_ang"],pre_meta["sat_nad_ang"])
    #print(post_meta["sat_north_ang"],pre_meta["sat_north_ang"])
    #print(post_meta["m_per_pix"],pre_meta["m_per_pix"])
    alpha = post_meta["sat_nad_ang"]-pre_meta["sat_nad_ang"];
    beta = post_meta["sat_north_ang"]-pre_meta["sat_north_ang"];
    scale = post_meta["m_per_pix"] / pre_meta["m_per_pix"]
    if(int(post_meta["org_w"])!=1024 or int(post_meta["org_h"])!=1024):
        print("No estamos conciderando imagenes cono zoom.!!!!")
    affine_transform = get_affine(0,0,1)
    print(affine_transform)
    merc_coords_list = []    
    #Encuentra las coordenadas mercator de la esquina inferior y superior de un rectángulo que contiene los polígonos.
    mercator = Proj(proj='merc')
    xmin, ymin, xmax, ymax = float('inf'), float('inf'), float('-inf'), float('-inf')
    for geo_poly in geo_poly_list:
        x_geo,y_geo = geo_poly.exterior.xy
        
        # Lista para almacenar las coordenadas alineadas
        aligned_coords = []
        
        # Aplicar la transformación afín a cada punto del polígono
        for x, y in zip(x_geo, y_geo):
            al_x, al_y = affine_transform * (x, y)
            aligned_coords.append((al_x, al_y))

        # Convertir las coordenadas alineadas a coordenadas Mercator
        
        x_merc, y_merc = mercator(*zip(*aligned_coords))
        #aligned_polygon = [affine_transform * (x,y) for x in x_merc for y in y_merc]
        merc_coords_list.append((x_merc,y_merc))
        
        # Calcula las coordenadas del bounding box
        pol_x_min, pol_y_min = np.min(x_merc), np.min(y_merc)
        pol_x_max, pol_y_max = np.max(x_merc), np.max(y_merc)
        
        xmin = min(xmin, pol_x_min)
        ymin = min(ymin, pol_y_min)
        xmax = max(xmax, pol_x_max)
        ymax = max(ymax, pol_y_max)
    
    zmin,zmax,wmin,wmax = find_corner_in_image(target_path)
    
    
    #print((zmin,wmin),(zmax,wmax))
    
    # Normaliza las coordenadas mercator a pixel teninedo en cuenta la posición determinada.
    cart_poly_list =[]
    for x_merc,y_merc in merc_coords_list:
        x_cart = (x_merc - xmin) * ((zmax-zmin) / (xmax-xmin)) + zmin
        y_cart = (y_merc - ymin) * ((wmax-wmin) / (ymax-ymin)) + wmin
        cart_poly_list.append(zip(x_cart,y_cart));
    
    return cart_poly_list
    
def create_aligned_polygons(pre_json_path : os.path,post_json_path : os.path,target_path):
    wkt_list : list;
    post_meta : dict;
    pre_meta: dict;
    with open(pre_json_path, 'r') as pre_j:
        pre_json = json.load(pre_j)
        meta = pre_json["metadata"]
        pre_meta = {
            "sat_nad_ang":meta['off_nadir_angle'],
            "sat_north_ang":meta['target_azimuth'],
            "org_w":meta['original_width'],
            "org_h":meta['original_height'],
            "m_per_pix":meta["gsd"]
            }

    with open(post_json_path, 'r') as post_j:
        post_json = json.load(post_j)
        meta = post_json["metadata"]
        post_meta = {
            "sat_nad_ang":meta['off_nadir_angle'],
            "sat_north_ang":meta['target_azimuth'],
            "org_w":meta['original_width'],
            "org_h":meta['original_height'],
            "m_per_pix":meta["gsd"]
            }
        wkt_list = [building["wkt"] for building in pre_json["features"]["lng_lat"]]
    
    geo_poly_list = [loads(c_wkt) for c_wkt in wkt_list]
    pixel_polygons = transform_polygons(geo_poly_list,target_path,pre_meta,post_meta)
    return pixel_polygons

def create_aligned_target_image(poly_list):
    image = np.zeros(shape=(1024,1024,3))
    # Crea la mascara y dibuja los polígonos en ella
    #colors = [plt.cm.Spectral(each) for each in np.linspace(0,1,num=len(poly_list))]
    colors = [plt.cm.Spectral(each) for each in [uniform(0,1) for i in range(len(poly_list))]]
    for cart_poly,c in zip(poly_list, colors):
        # Transformación que invierte la imagen para que el 0,0 este en el borde inferior izquierdo
        tranform = from_origin(0,1024,1,1) 
        pts = np.array([tranform*(x,y) for x,y in cart_poly], dtype=np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.fillPoly(image, [pts], color=c) 
    
    scale = 0.5
    alpha = np.deg2rad(60)
    beta = np.deg2rad(0)
    # Construir la matriz de transformación afín 2x3
    transform_matrix = np.array([[scale*np.cos(beta),-np.sin(beta),0],
                              [np.sin(beta),scale*np.cos(beta)*np.cos(alpha),-np.sin(alpha)]],np.float32)
    new_im = cv2.warpAffine(image, transform_matrix, (1024, 1024))
    plt.imshow(new_im)
    plt.show()
    return image

def create_aligned_labels(poly_list,json_path):
    return None

pair : DisasterPair
for i,pair in enumerate(disaster_pair_dicc.values()):
    
    pre_json_path = os.path.join(label_dir, pair.get_pre().get_json_file_name());
    post_json_path = os.path.join(label_dir, pair.get_post().get_json_file_name());

    post_target_path = os.path.join(target_dir,pair.get_post().get_target_file_name());
    
    aligned_polygons = create_aligned_polygons(pre_json_path,post_json_path,post_target_path)
    post_aligned_mask = create_aligned_target_image(aligned_polygons)
    post_aligned_json = create_aligned_labels(aligned_polygons,post_json_path)
    # Guardar json y mascara
    # cv2.imwrite(os.path.join(masked_target_dir,pair.get_pre().get_target_file_name()),post_aligned_mask)
    print(f"Progreso: {(i+1)}/{len(disaster_pair_dicc.values())}", end='\r')

In [ ]:
from skimage import io, color, transform
from skimage.feature import match_template
import math
import numpy as np

folder = folder_dict["guatemala-volcano_00000000"]
# Cargar las imágenes
p = os.path.join(folder.get_folder_path(),folder.get_pre());
img_pre_disaster = cv2.imread(p)
p = os.path.join(folder.get_folder_path(),folder.get_post());
img_post_disaster = cv2.imread(p)
# Cargar los targets
p = os.path.join(folder.get_folder_path(),folder.get_instance_mask());
target_pre_disaster_img = cv2.imread(p)
p = os.path.join(folder.get_folder_path(),folder.get_class_mask());
target_post_disaster_img = cv2.imread(p)
# Bounding box
p = os.path.join(folder.get_folder_path(),folder.get_bbox());
bounding_boxes = pd.read_csv(p)  # Lista de bounding boxes (x, y, width, height)


# Convertir a escala de grises (opcional, dependiendo de las imágenes)
image1_gray = color.rgb2gray(img_pre_disaster)
image2_gray = color.rgb2gray(img_post_disaster)

# Realizar la registración utilizando template matching
from scipy.signal import correlate2d

result = correlate2d(image1_gray, image2_gray, mode='same')

#result = match_template(image1_gray, image2_gray)
# Encontrar la mejor coincidencia
y, x = np.unravel_index(np.argmax(result), result.shape)

# Alinear la segunda imagen con la primera
image2_aligned = transform.warp(img_post_disaster, transform.AffineTransform(translation=(-x, -y)))

# Guardar la imagen alineada
#print(img_pre_disaster.shape,image2_aligned.shape)
aligned_image = np.clip(image2_aligned*255, 0, 255).astype(dtype=np.uint8)
showImage(img_pre_disaster,aligned_image,target_pre_disaster_img, target_post_disaster_img, bounding_boxes)